In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
with open(r"F:\projects\PlayGround\gsoft\data\names.txt") as f:
    words = f.read().splitlines()

In [3]:
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))
stoi  = {ch:i+1 for i,ch in enumerate(chars)}
stoi['.'] = 0
itos = {i:ch for ch,i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
block_size = 3

def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and appended
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [16]:
# MLP

n_embed = 10
n_hidden2 = 200
n_hidden3 = 100



g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed),               generator=g)
W1 = torch.randn((n_embed * block_size, n_hidden2),  generator=g) * 0.01
b1 = torch.randn(n_hidden2,                          generator=g) * 0
W2 = torch.randn((n_hidden2, n_hidden3),             generator=g) * 0.01
b2 = torch.randn(n_hidden3,                          generator=g) * 0
W3 = torch.randn((n_hidden3, vocab_size),            generator=g) * 0.01
b3 = torch.randn(vocab_size,                         generator=g) * 0


parameters = [C, W1, b1, W2, b2, W3, b3]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

29297


In [17]:
W2.shape

torch.Size([200, 100])

In [18]:
#optimization
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    
    #mini_batch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] #batch X,Y
    
    # forward pass
    emb = C[Xb]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    h1 = torch.tanh(hpreact)
    h2preact = h1 @ W2 + b2
    h2 = torch.tanh(h2preact)
    logits = h2 @ W3 + b3
    loss = F.cross_entropy(logits, Yb)
    
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    #update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
        
    #track stats
    if i % 10000 == 0:
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())


      0/ 200000: 3.2960
  10000/ 200000: 1.9153
  20000/ 200000: 2.2974
  30000/ 200000: 2.1686
  40000/ 200000: 2.0499
  50000/ 200000: 2.3161
  60000/ 200000: 2.1599
  70000/ 200000: 2.2473
  80000/ 200000: 2.4042
  90000/ 200000: 2.0684
 100000/ 200000: 2.0316
 110000/ 200000: 2.3115
 120000/ 200000: 1.8919
 130000/ 200000: 2.5434
 140000/ 200000: 2.1427
 150000/ 200000: 1.7221
 160000/ 200000: 2.1433
 170000/ 200000: 1.9160
 180000/ 200000: 1.8292
 190000/ 200000: 2.1002


In [21]:
@torch.no_grad()
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte)
    }[split]
    
    emb = C[x]
    embcat = emb.view(emb.shape[0], -1)
    h = torch.tanh(embcat @ W1 + b1)
    h1 = torch.tanh(h @ W2 + b2)
    logits = h1 @ W3 + b3
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())
    
split_loss('train')
split_loss('val')   

train 2.0204505920410156
val 2.097294569015503


In [22]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        h1 = torch.tanh(h @ W2 + b2)
        logits = h1 @ W3 + b3
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(itos[ix])
        if ix == 0:
            break
    print("".join(out))

mon.
almyah.
seel.
ndhanalee.
thruthadrie.
caileed.
eliah.
miloparekeison.
anara.
mykimson.
ami.
shub.
roshira.
sten.
joselynn.
novana.
ubusted.
ryyah.
fael.
yuva.
